In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import functions as st
import os
import pickle

parcellation = 'Sch240'
n_connectomes = '20'
behaviour_list = ['APM','NART_IQ','Q1_TotalWords','Q6_TotalCorrect','CoC_abs_rev']
#normalise_behav = True
#reverse_behav = True
regress_size = False
CCApermutations = 1000
MCA_components = 5
CCA_components = 5
outdir = '/Users/luke/Documents/Projects/StrokeNet/Docs/Figures/2020/'

In [8]:
behav_df, CM = st.load_data(parcellation=parcellation,
                         n_connectomes=n_connectomes,
                         behaviour_list=behaviour_list,
                              load_nifti=False)

#MNIcoords, networks, network_labels = st.get_atlas_info(parcellation=parcellation)

136it [00:06, 19.48it/s]


## prepare out of loop data

In [4]:
# reshape connectivity data
n_subs = np.shape(CM['Diff'])[2]
n_nodes = np.shape(CM['Diff'])[0]
n_edges = np.int((n_nodes*(n_nodes-1))/2)
print('Subjects=',n_subs,', nodes=',n_nodes,', edges=',n_edges)

X_full = np.zeros((n_subs,n_edges))
index_upper = np.triu_indices(n_nodes, k=1)
for subj in range(n_subs):
    data = CM['Diff'][:,:,subj].copy()
    X_full[subj,:] = data[index_upper].copy()

# binarize edges
X_full = X_full > 0

# remove non informative features
keep_features = np.sum(X_full,axis=0)>3
X = X_full[:,keep_features]
print('Edges included=',np.sum(keep_features))

Subjects= 80 , nodes= 240 , edges= 28680
Edges included= 7779


In [5]:
y = behav_df[behaviour_list].values
y = y * -1 

## Do the prediction

In [22]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.cross_decomposition import CCA
from scipy.stats import spearmanr
from tqdm import tqdm
import prince

#number of folds
num_cvs = 5

#number of iterations
cv_iterations = 2

# define cross validation scheme
cv = KFold(n_splits=num_cvs,shuffle=True)

#output variables
r_avg = np.zeros((cv_iterations,np.shape(y)[1]))
rs_avg = r_avg.copy()
MAE_avg = r_avg.copy()
Rsqr_avg = r_avg.copy()

for j in tqdm(range(cv_iterations)):
    
    # variables for this iteration
    r = np.zeros((num_cvs,np.shape(y)[1]))
    rs = r.copy()
    MAE = r.copy()
    Rsqr = r.copy()

    cv_count = 0
    for train_index, test_index in cv.split(X):
        
        #organise the data into training and testing sets
        X_train = X[train_index,:]
        X_test  = X[test_index,:]
        
        y_train = y[train_index,:]
        y_test = y[test_index,:]
        
        # trim X to informative features (X train needs to be applicable to X test)
        
        
        # normalise y in train set
        scaler = preprocessing.QuantileTransformer(n_quantiles=len(y_train)).fit(y_train)
        y_train_scaled = scaler.transform(y_train)
        
        # and apply normalization to test set
        y_test_scaled = scaler.transform(y_test)
        
        # perform MCA on x in train set
        mca = prince.MCA(n_components=MCA_components).fit(X_train)

        # individual loadings onto components
        mca.ind_scores = mca.row_coordinates(X).values

        # and apply MCA to test set
        
#         # do cca
#         cv_cca = CCA(n_components=1)
#         cv_cca.fit(X_train,y_train)

#         # predict left out values
#         y_pred = cv_cca.predict(X_test)

#         # save predictions for this cv
#         for i in range(np.shape(y)[1]):
#             r[cv_count,i] = np.corrcoef(y[test_index,i],y_pred[:,i])[0,1]
#             rs[cv_count,i] = spearmanr(y[test_index,i],y_pred[:,i])[0]
#             MAE[cv_count,i] = mean_absolute_error(y[test_index,i],y_pred[:,i])
#             Rsqr[cv_count,i] = r2_score(y[test_index,i],y_pred[:,i])
#         cv_count = cv_count+1 

#     # average predictions for this iteration
#     r_avg[j,:] = np.mean(r,axis=0)
#     rs_avg[j,:] = np.mean(rs,axis=0)
#     MAE_avg[j,:] = np.mean(MAE,axis=0)
#     Rsqr_avg[j,:] = np.mean(Rsqr,axis=0)
# return r_avg,rs_avg,MAE_avg,Rsqr_avg

  0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Found unknown categories [True] in column 1035 during transform

In [21]:
mca.

array([[0.61111111, 0.6984127 , 0.98891884, 0.86507937, 0.71428571],
       [0.03968254, 0.51587302, 0.26984127, 0.14285714, 0.00497713],
       [0.61111111, 0.49206349, 0.88095238, 0.57936508, 0.36507937],
       [0.        , 0.07142857, 0.03968254, 0.04761905, 0.73809524],
       [0.23015873, 0.37301587, 0.55026455, 0.73809524, 0.23809524],
       [0.61111111, 0.75396825, 0.41587302, 0.43650794, 0.8042328 ],
       [0.46825397, 0.42063492, 0.87037037, 0.8968254 , 0.17777778],
       [1.        , 0.75396825, 0.55555556, 0.51587302, 1.        ],
       [0.31746032, 0.31746032, 0.22222222, 0.19047619, 0.62857143],
       [0.46825397, 0.83333333, 0.71428571, 0.84126984, 0.76190476],
       [1.        , 0.46031746, 0.9047619 , 0.80952381, 1.        ],
       [0.12698413, 0.37301587, 0.42857143, 0.65079365, 0.26839827],
       [0.03968254, 0.37301587, 0.3968254 , 0.19047619, 0.56613757],
       [1.        , 0.42063492, 0.82010582, 0.53968254, 0.47619048],
       [0.12698413, 0.96825397, 0.